In [1]:
import pandas as pd
import sqlalchemy as sql

In [2]:
recessions={
    'savings_and_loans_crisis': pd.date_range('1990-07', '1991-03'), 
    'dot_com_bust': pd.date_range('2001-03', '2001-11'), 
    'great_recession': pd.date_range('2007-12', '2009-06'), 
    'covid': pd.date_range('2020-04', '2020-06'), 
    # subject to change
    'current': pd.date_range('2022-01', '2022-08-09')
}

In [3]:
engine=sql.create_engine('sqlite:///recession_analysis.db')

In [30]:
sql_query=f'select * from price_history'
price_history=pd.read_sql(sql_query, con=engine, index_col='Date', parse_dates='Date')

price_history=price_history.pivot(columns='ticker', values='Close').pct_change()
price_history.head()

ticker,IYT,MSCI,SPY,VENAX,VGHCX,^IXIC
Date,,,,,,
1971-02-05,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-08,NaN,NaN,NaN,NaN,NaN,0.008400
1971-02-09,NaN,NaN,NaN,NaN,NaN,-0.000793
1971-02-10,NaN,NaN,NaN,NaN,NaN,-0.000695
1971-02-11,NaN,NaN,NaN,NaN,NaN,0.007548


In [31]:
post_recessions={}
for recession_name, recession_dates in recessions.items(): 
    post_recessions[recession_name]=pd.date_range(start=recession_dates[-1], periods=30)
post_recessions

{'savings_and_loans_crisis': DatetimeIndex(['1991-03-01', '1991-03-02', '1991-03-03', '1991-03-04',
                '1991-03-05', '1991-03-06', '1991-03-07', '1991-03-08',
                '1991-03-09', '1991-03-10', '1991-03-11', '1991-03-12',
                '1991-03-13', '1991-03-14', '1991-03-15', '1991-03-16',
                '1991-03-17', '1991-03-18', '1991-03-19', '1991-03-20',
                '1991-03-21', '1991-03-22', '1991-03-23', '1991-03-24',
                '1991-03-25', '1991-03-26', '1991-03-27', '1991-03-28',
                '1991-03-29', '1991-03-30'],
               dtype='datetime64[ns]', freq='D'),
 'dot_com_bust': DatetimeIndex(['2001-11-01', '2001-11-02', '2001-11-03', '2001-11-04',
                '2001-11-05', '2001-11-06', '2001-11-07', '2001-11-08',
                '2001-11-09', '2001-11-10', '2001-11-11', '2001-11-12',
                '2001-11-13', '2001-11-14', '2001-11-15', '2001-11-16',
                '2001-11-17', '2001-11-18', '2001-11-19', '2001-11-20

In [32]:
for recession_name, recession_dates in post_recessions.items(): 
    price_history.loc[price_history.index.isin(recession_dates), 'event']=recession_name
price_history['event'].value_counts()

great_recession             22
covid                       22
dot_com_bust                21
savings_and_loans_crisis    20
current                      1
Name: event, dtype: int64

In [34]:
price_history
#.pivot_table(columns='event', aggfunc='cumprod')

ticker,IYT,MSCI,SPY,VENAX,VGHCX,^IXIC,event
Date,,,,,,,
1971-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-08,NaN,NaN,NaN,NaN,NaN,0.008400,NaN
1971-02-09,NaN,NaN,NaN,NaN,NaN,-0.000793,NaN
1971-02-10,NaN,NaN,NaN,NaN,NaN,-0.000695,NaN
1971-02-11,NaN,NaN,NaN,NaN,NaN,0.007548,NaN
...,...,...,...,...,...,...,...
2022-08-03,0.014626,0.039021,0.015659,-0.027208,0.015799,0.025865,NaN
2022-08-04,0.008564,0.000866,-0.000676,-0.036782,0.003312,0.004138,NaN
2022-08-05,0.006137,-0.001168,-0.001690,0.021082,0.001196,-0.004955,NaN
